In [1]:
import sys
import os
sys.path.append(os.path.abspath("..")) 

import numpy as np
import pandas as pd
import re

from scripts.get_paths import get_path

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
paths = get_path()

In [3]:
df = pd.read_csv(paths.features_2 / "extracted_features.csv", index_col=False)

### Extract gaze_openface (lookind away from screen) from gaze_angle_x and gaze_angle_y

In [4]:
from scripts.mixins import FeaturesMixin
try:
    from eyelink.objects.geometry import ScreenSetup, \
        CameraSetup, GeometrySetup
except Exception:
    from collections import namedtuple
    GeometrySetup = namedtuple('GeometrySetup', ['vfov', 'hfov'])
from scripts.eye_gaze import AversionExtractor

In [5]:
# --------- geometry setup for the lab setting used for this study ---------
try:
    screen = ScreenSetup(
        width=309.9,
        height=174.5,
        resolution_width=1920,
        resolution_height=1080
    )
    camera = CameraSetup(
        vertical_distance_to_monitor=7,  # approximately
        horizontal_distance_to_camera=309.9 / 2
    )
    geometry = GeometrySetup(
        screen_setup=screen,
        camera_setup=camera,
        distance_to_top=600,
        distance_to_bottom=620,  # approximately
    )
except Exception:
    geometry = GeometrySetup(vfov=0.28880, hfov=0.50545)


In [6]:
dvpfe = AversionExtractor(geometry)
df = dvpfe.fit_transform(df)
df['gaze_openface'] = 1 - df['view_looks_at_display']
df.columns

Index(['frame', 'pitch_pyafar', 'yaw_pyafar', 'roll_pyafar',
       'smile_pyafar_pre', 'Occ_au_4', 'Occ_au_6', 'participant_id', 'clip',
       'timestamp', 'confidence', 'gaze_angle_x', 'gaze_angle_y',
       'pitch_openface', 'yaw_openface', 'roll_openface', 'smile_openface',
       'AU12_r', 'gaze_pyafar_pre', 'label_id', 'video_id', 'Id', 'NT', 'part',
       'speaker', 'gaze_fd', 'smile_fd', 'gaze_vp', 'smile_vp', 'nod_fd',
       'nod_vp', 'nod_lk', 'gaze_agreed', 'smile_agreed', 'nod_agreed',
       'view_looks_at_display', 'gaze_openface'],
      dtype='object')

## Get Nods for pyafar and openface

In [7]:
from scripts.nod_detector import NodDetector
df = df.sort_values(['participant_id', 'clip', 'frame'], kind='mergesort')

df['nod_openface'] = 0
df['nod_pyafar'] = 0

for (participant_id, clip), df_sub in df.groupby(['participant_id', 'clip'], sort=False):

    fps = (df_sub['frame'].iloc[-1] - df_sub['frame'].iloc[0]) / (
            df_sub['timestamp'].iloc[-1] - df_sub['timestamp'].iloc[0])

    # --- OPENFACE --- 1,4 selected using grid search for max agreement on NT and reduced clips
    detector = NodDetector(k_std_velocity=1.0,k_std_still_pitch=4.0)

    pitch_openface = df_sub["pitch_openface"].to_numpy()
    yaw_openface = df_sub["yaw_openface"].to_numpy()

    # If detector can’t handle NaNs, skip or fill:
    if np.isnan(pitch_openface).any() or np.isnan(yaw_openface).any():
        pitch_openface = pd.Series(pitch_openface).interpolate(limit_direction="both").to_numpy()
        yaw_openface   = pd.Series(yaw_openface).interpolate(limit_direction="both").to_numpy()

    events_openface, _ = detector.detect_nod(
        pitch=pitch_openface,
        yaw=yaw_openface,
        fps=fps,
        radians=True
    )
    for start, end in events_openface:
        idx = df_sub.index[start:end+1]   
        df.loc[idx, 'nod_openface'] = 1

    # --- PYAFAR --- 1,4 selected using grid search for max agreement on NT and reduced clips
    detector = NodDetector(k_std_velocity=1.0,k_std_still_pitch=4.0)
    pitch_pyafar = df_sub["pitch_pyafar"].to_numpy()
    yaw_pyafar = df_sub["yaw_pyafar"].to_numpy()

    # If detector can’t handle NaNs, skip or fill:
    if np.isnan(pitch_pyafar).any() or np.isnan(yaw_pyafar).any():
        pitch_pyafar = pd.Series(pitch_pyafar).interpolate(limit_direction="both").to_numpy()
        yaw_pyafar   = pd.Series(yaw_pyafar).interpolate(limit_direction="both").to_numpy()

    events_pyafar, _ = detector.detect_nod(
        pitch=pitch_pyafar,
        yaw=yaw_pyafar,
        fps=fps,
        radians=False
    )
    for start, end in events_pyafar:
        idx = df_sub.index[start:end+1]
        df.loc[idx, 'nod_pyafar'] = 1


In [8]:
df.columns

Index(['frame', 'pitch_pyafar', 'yaw_pyafar', 'roll_pyafar',
       'smile_pyafar_pre', 'Occ_au_4', 'Occ_au_6', 'participant_id', 'clip',
       'timestamp', 'confidence', 'gaze_angle_x', 'gaze_angle_y',
       'pitch_openface', 'yaw_openface', 'roll_openface', 'smile_openface',
       'AU12_r', 'gaze_pyafar_pre', 'label_id', 'video_id', 'Id', 'NT', 'part',
       'speaker', 'gaze_fd', 'smile_fd', 'gaze_vp', 'smile_vp', 'nod_fd',
       'nod_vp', 'nod_lk', 'gaze_agreed', 'smile_agreed', 'nod_agreed',
       'view_looks_at_display', 'gaze_openface', 'nod_openface', 'nod_pyafar'],
      dtype='object')

In [9]:
df.to_csv(paths.features_2 / "extracted_features.csv", index=False)